In [62]:
# encoding: utf-8

import pandas as pd
from pandas import *
import datetime
import json
from pymongo import MongoClient
from collections import defaultdict

pd.set_option('display.width', None)  # 设置字符显示宽度
pd.set_option('display.max_rows', None)  # 设置显示最大行
pd.set_option('display.max_columns', None)  # 设置显示最大行

client = MongoClient('localhost', 27017)
db = client.futures
indexMarket = db.indexMarket
peak = db.peak
unit=db.unit

start='20190601'
# var='JD'

indexMarket = DataFrame(list(indexMarket.find({'date': {'$gte': start}})))

unit = DataFrame(list(unit.find()))
dd=unit['variety']
# dd

# for i in dd:
#     df=indexMarket[indexMarket['variety']==i]
# df

In [63]:
for i in dd:
    df=indexMarket[indexMarket['variety']==i]
# df=pd.DataFrame(df)
# print(df.tail(5))
    df= df[['date', 'variety','set_open','set_close', 'set_high', 'set_low']]

# df['date'] = pd.to_datetime(df['date'])
    df.set_index('date',inplace=True)
#     df.tail(2)
    maxs=df[['set_high','set_low']].stack().max()
    mins=df[['set_high','set_low']].stack().min()
    print(i)
    print(maxs,mins)



IF
3989.0 3571.0
IH
3043.0 2718.0
IC
5256.0 4514.0
TF
100.0 99.0
T
100.0 98.0
TS
100.0 100.0
CU
48367.0 45743.0
AU
364.0 315.0
AG
4845.0 3691.0
ZN
19769.0 18265.0
AL
14657.0 13708.0
RU
12271.0 10556.0
RB
4027.0 3245.0
FU
3000.0 2075.0
HC
3912.0 3239.0
BU
3429.0 2884.0
PB
17526.0 16047.0
NI
149080.0 105445.0
SN
145088.0 127830.0
WR
4574.0 3598.0
SC
496.0 402.0
A
3642.0 3314.0
B
3475.0 2999.0
BB
185.0 146.0
C
1964.0 1823.0
CS
2380.0 2178.0
FB
73.0 56.0
I
878.0 570.0
J
2221.0 1731.0
JD
4804.0 4105.0
JM
1423.0 1203.0
L
7876.0 7033.0
M
3057.0 2748.0
P
5173.0 4221.0
PP
8801.0 7762.0
V
6892.0 6276.0
Y
6322.0 5428.0
WH
2479.0 2119.0
PM
2398.0 2055.0
CF
13610.0 12010.0
CY
21941.0 19805.0
SR
5634.0 5104.0
TA
5978.0 4821.0
OI
7583.0 6899.0
RI
2980.0 2326.0
MA
2431.0 2062.0
FG
1518.0 1373.0
RS
4131.0 3457.0
RM
2485.0 2234.0
ZC
595.0 552.0
JR
3441.0 2802.0
LR
2894.0 2459.0
SF
6449.0 5711.0
SM
7604.0 6278.0
AP
9769.0 7355.0
EG
5336.0 4254.0
CJ
11546.0 9780.0
SP
4867.0 4433.0


In [65]:
maxs=df[['set_high','set_low']].stack().max()
mins=df[['set_high','set_low']].stack().min()
print(maxs,mins)

4867.0 4433.0


In [66]:
gains=round(((maxs/mins-1)*100),2)
gains

9.79

In [67]:
lesses=round(((1-mins/maxs)*100),2)
lesses

8.92

In [68]:
# df['one'] = df["one"].apply(lambda x: x*10 if x<2 else (x**2 if x<4 else x+10))

gains=round(((maxs/mins-1)*100),2)
lesses=round(((1-mins/maxs)*100),2)

df['peak']=(lambda x:gains if mins < df['set_close'][-1] else lesses)(1)
# print(peak(1))
# df

In [69]:
# if mins>df['set_close'][-1]:
#     print(gains)

# else:
#     print(lesses)

In [70]:
# date=df.append([maxs],[peak(1)])#.fillna(method='ffill')#,columns=['peak']
# print(date)

In [81]:
df2 = DataFrame()
for i in dd:
    df=indexMarket[indexMarket['variety']==i]
# df=pd.DataFrame(df)
# print(df.tail(5))
    date=df[['date'][-1]]

    data= df[['date', 'variety','set_open','set_close', 'set_high', 'set_low']]
# df
# df['date'] = pd.to_datetime(df['date'])
    data.set_index('date',inplace=True)
#     ss=data['set_close']
# ss
#     df.tail(2)
    maxs=data[['set_high','set_low']].stack().max()
# maxs
    mins=data[['set_high','set_low']].stack().min()
    gains=round(((maxs/mins-1)*100),2)
    lesses=round(((1-mins/maxs)*100),2)

    peak=(lambda x:gains if mins < data['set_close'][-1] else lesses)(1)
# peak
#     df['peak']=(lambda x:gains if mins < df['set_close'][-1] else lesses)(1)
#     df=df[-1:]
# df
#     peak=peak(1)
#     df2=df2.append(pd.DataFrame([maxs],[date]))
#     df2=df2.append({'maxs':maxs,'mins':mins,'gains':gains,'lesses':lesses},date)
    columns=list(['variety','new_close','maxs','mins','gains','lesses'])
    data2 = pd.DataFrame([[i,data['set_close'][-1],maxs,mins,gains,lesses]],date, columns)[-1:]

#     data2=pd.DataFrame({'maxs':maxs,'mins':mins,'gains':gains,'lesses':lesses,'peak':peak},date)[-1:]
#     data2=df2.append(data2)
data2


,variety,new_close,maxs,mins,gains,lesses
date,,,,,,
20191025,SP,4632.0,4867.0,4433.0,9.79,8.92


In [86]:
# df2=pd.DataFrame()
# df3={}
for i in set(dd):
    try:
        df=indexMarket[indexMarket['variety']==i]
#         print(df)
        date=df[['date'][-1]]
        data= df[['date', 'variety','set_open','set_close', 'set_high', 'set_low']]
        data.set_index('date',inplace=True)
        maxs=data[['set_high','set_low']].stack().max()
        mins=data[['set_high','set_low']].stack().min()
        
        gains=round(((maxs/mins-1)*100),2)
        lesses=round(((1-mins/maxs)*100),2)
        peak=(lambda x:gains if mins < data['set_close'][-1] else lesses)(1)
#         print(peak)
        columns=list(['variety','new_close','maxs','mins'])
        data2 = pd.DataFrame([[i,data['set_close'][-1],maxs,mins],date,columns])[-1:]
        print(data2)
#         data2=df2.append(data2)
#         
#         data2 = data2.reset_index()
        
        
#         data=data[['variety','maxs','mins','set_close','peak']]

#         data=data[-1:]
# #         data=pd.concat(data)
# #         data.append(data)
#         print(data)
        
        
    except:
        pass
        continue
# for key, value in data2.items():
#             print(key)
#         data2[key].append[]???
        
                              
    



In [73]:
df2=pd.DataFrame()
for i in set(dd):
    try:
        df=indexMarket[indexMarket['variety']==i]
        date=df[['date'][-1]]
        data= df[['date', 'variety','set_open','set_close', 'set_high', 'set_low']]
        data.set_index('date',inplace=True)
        maxs=data[['set_high','set_low']].stack().max()
        mins=data[['set_high','set_low']].stack().min()
        gains=round(((maxs/mins-1)*100),2)
        lesses=round(((1-mins/maxs)*100),2)
        peak=(lambda x:gains if mins < data['set_close'][-1] else lesses)(1)
        columns=list(['variety','new_close','maxs','mins'])
#         data2 = pd.DataFrame([[i,data['set_close'][-1],maxs,mins],date,columns])[-1:]

        data=data.copy()
        
        data['maxs']=maxs
        data['mins']=mins
        data['peak']=peak
        data=data[-1]
        print(data)
        
        
    except:
        pass
#         continue


In [112]:
for i in set(dd):
    try:
        df=indexMarket[indexMarket['variety']==i]
#         date=df[['date'][-1]]
        df= df[['date', 'variety','set_open','set_close', 'set_high', 'set_low']]
        df.set_index('date',inplace=True)
        maxs=df[['set_high','set_low']].stack().max()
        mins=df[['set_high','set_low']].stack().min()
        gains=round(((maxs/mins-1)*100),2)
        lesses=round(((1-mins/maxs)*100),2)
        peak=(lambda x:gains if mins < data['set_close'][-1] else lesses)(1)
        df2=df.copy()
        df2['maxs']=maxs
        df2['mins']=mins
        df2['peak']=peak
#         df2 = df2.reset_index()
        df2=df2[['variety','maxs','mins','set_close','peak']][-1:]
#         df2['date']=df
#         
#         df2=pd.DataFrame(df2)
    
#         df2=peak.insert(df2)
#         print(df2)
        peak.insert(json.loads(df2.T.to_json()).values())
        print(json.loads(df2.T.to_json()).values())
        print(1)
    except:
        print('error')


error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
